In [1]:

import librosa
import librosa.display
import matplotlib.pyplot as plt
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, TIT2, TPE1, TALB, TCOM, COMM, TCON, TDRC, TPOS, TXXX, APIC
import numpy as np

def extract_metadata(audio_file):
    # Open the audio file
    audio = MP3(audio_file, ID3=ID3)

    # Extract metadata
    metadata = {}
    id3 = audio.tags
    if id3:
        metadata['album'] = id3.get('TALB', [''])[0]
        metadata['albumartist'] = id3.get('TPE2', [''])[0]
        metadata['artist'] = id3.get('TPE1', [''])[0]
        
        # Extract comment
        comment = ''
        for key in id3.keys():
            if key.startswith('COMM'):  # Check for COMMENT tag
                comment = id3[key].text[0]
                break
            elif key.startswith('COMM:'):  # Check for additional encoding information
                comment = id3[key].text[0]
                break
            elif key.startswith('TXXX:'):  # Check for user-defined text tag
                if 'comment' in key.lower():
                    comment = id3[key].text[0]
                    break
        metadata['comment'] = comment
        
        metadata['compilation'] = id3.get('TCMP', [''])[0]
        metadata['composer'] = id3.get('TCOM', [''])[0]
        metadata['discnumber'] = id3.get('TPOS', [''])[0]
        metadata['genre'] = id3.get('TCON', [''])[0]
        metadata['lyrics'] = id3.get('USLT', [''])[0]
        metadata['totaldiscs'] = id3.get('TXXX:totaldiscs', [''])[0]
        metadata['totaltracks'] = id3.get('TXXX:totaltracks', [''])[0]
        metadata['tracknumber'] = id3.get('TRCK', [''])[0]
        metadata['tracktitle'] = id3.get('TIT2', [''])[0]
        metadata['year'] = id3.get('TDRC', [''])[0]
        metadata['isrc'] = id3.get('TSRC', [''])[0]

        # Extract artwork
        if 'APIC:' in id3:
            artwork = id3['APIC:'].data
            metadata['artwork'] = artwork
        
    return metadata

extract_metadata("000002.mp3")

{'album': 'Free For All Comp LP',
 'albumartist': 'Various Artists',
 'artist': 'Token Entry',
 'comment': 'http://www.myspace.com/awayoflifemusic  http://creativecommons.org/licenses/by-nc-sa/3.0/us/',
 'compilation': '',
 'composer': '',
 'discnumber': '',
 'genre': 'Hip-Hop',
 'lyrics': '',
 'totaldiscs': '',
 'totaltracks': '',
 'tracknumber': '03',
 'tracktitle': 'The Edge',
 'year': '2008',
 'isrc': ''}

In [3]:

import librosa
# Load the audio file
audio_file = "000002.mp3"
y, sr = librosa.load(audio_file)

# Compute MFCCs
mfccs = librosa.feature.mfcc(y=y, sr=sr)

# Compute spectral centroid
spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

# Compute zero-crossing rate
zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
# MFCCs: MFCCs are like a fingerprint for sounds,
# capturing their unique characteristics such as pitch and timbre, which helps computers understand and differentiate between different sounds.

# Visualize MFCCs
#plt.figure(figsize=(10, 6))
#librosa.display.specshow(mfccs, x_axis='time')
#plt.colorbar()
#plt.title('MFCC')
#plt.tight_layout()

# Spectral Centroid: Spectral centroid is like the average frequency of a sound, telling us whether
# it's high-pitched or low-pitched.

# Visualize Spectral Centroid
#plt.figure(figsize=(10, 6))
#plt.semilogy(spectral_centroid.T, label='Spectral Centroid')
#plt.ylabel('Hz')
#plt.xticks([])
#plt.xlim([0, spectral_centroid.shape[-1]])
#plt.title('Spectral Centroid')
#plt.tight_layout()

# Zero-Crossing Rate: Zero-crossing rate is like counting how many times the sound wave crosses the 
# middle line (zero amplitude), which gives us an idea of how "noisy" or "smooth" the sound is.

# Visualize Zero-Crossing Rate
#plt.figure(figsize=(10, 6))
#plt.plot(zero_crossing_rate.T, label='Zero Crossing Rate')
#plt.ylabel('Zero-Crossing Rate')
#plt.xlabel('Frame')
#plt.title('Zero Crossing Rate')
#plt.tight_layout()

#plt.show()

# Compute statistics for MFCCs
mfccs_stats = {
    "mean": np.mean(mfccs, axis=1),
    "median": np.median(mfccs, axis=1),
    "std": np.std(mfccs, axis=1),
    # Add more statistics as needed
}

# Compute statistics for spectral centroid
spectral_centroid_stats = {
    "mean": np.mean(spectral_centroid),
    "median": np.median(spectral_centroid),
    "std": np.std(spectral_centroid),
}

# Compute statistics for zero-crossing rate
zero_crossing_rate_stats = {
    "mean": np.mean(zero_crossing_rate),
    "median": np.median(zero_crossing_rate),
    "std": np.std(zero_crossing_rate),
}

# Print statistics
print("MFCCs Statistics: "+ audio_file)
for stat, values in mfccs_stats.items():
    print(f"{stat}: {values}")

print("\nSpectral Centroid Statistics: "+ audio_file)
for stat, value in spectral_centroid_stats.items():
    print(f"{stat}: {value}")

print("\nZero-Crossing Rate Statistics: "+ audio_file)
for stat, value in zero_crossing_rate_stats.items():
    print(f"{stat}: {value}")

AttributeError: module 'soundfile' has no attribute 'SoundFileRuntimeError'

In [ ]:
from pymongo import MongoClient
from gridfs import GridFS
client = MongoClient('mongodb://localhost:27017/')
db = client['Dataset']
with open('000002.mp3', 'rb') as mp3_file:
    mp3_content = mp3_file.read()
fs = GridFS(db)
file_id = fs.put(mp3_content, filename='000002.mp3')
print("File saved with ID:", file_id)


File saved with ID: 6639d3451aab3ce7c391600b


In [ ]:
# Iterate over the cursor and print each document
cursor = db.fs.files.find()
for document in cursor:
    print(document)


{'_id': ObjectId('6639d3451aab3ce7c391600b'), 'filename': '000002.mp3', 'chunkSize': 261120, 'length': 961014, 'uploadDate': datetime.datetime(2024, 5, 7, 7, 7, 49, 574000)}


In [3]:
import pymongo

# MongoDB connection settings
mongodb_url = 'mongodb://localhost:27017'
database_name = 'Dataset'
collection_name = 'fs.chunks'

# Connect to MongoDB
client = pymongo.MongoClient(mongodb_url)
db = client[database_name]
collection = db[collection_name]

# Define a function to fetch the Base64 data
def fetch_base64_data():
    # Find any document in the collection
    document = collection.find_one()
    #print(document)
    if document:
        # Print the value of the .data field
        data_value = document.get('data')
        print('.data field value:', data_value)
        # Output file path
        output_file_path = 'output.mp3'

        # Write the binary data to an MP3 file
        with open(output_file_path, 'wb') as output_file:
            output_file.write(data_value)

        print(f'MP3 file saved: {output_file_path}')
    else:
        print('No document found in the collection.')

# Call the function to fetch the Base64 data
fetch_base64_data()


.data field value: b'ID3\x04\x00\x00\x00\x00\x02\x03TCON\x00\x00\x00\t\x00\x00\x03Hip-Hop\x00TRCK\x00\x00\x00\x03\x00\x00\x033\x00TIT2\x00\x00\x00\x06\x00\x00\x03Food\x00TXXX\x00\x00\x00f\x00\x00\x03comment\x00http://www.myspace.com/awayoflifemusic  http://creativecommons.org/licenses/by-nc-sa/3.0/us/\x00TPE1\x00\x00\x00\x06\x00\x00\x03AWOL\x00TALB\x00\x00\x00\x16\x00\x00\x03AWOL - A Way Of Life\x00TDRC\x00\x00\x00\x06\x00\x00\x032008\x00TSSE\x00\x00\x00\x0f\x00\x00\x03Lavf57.63.100\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xfb\xd0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Info\x00\x00\x00\x0f\x00\x00\x04|\x00\x0e\xa7\xd5\x00\x02\x05\x08\n\r\x10\x12\x14\x17\x1a\x1c\x1f"$\')+.1369;=@CEHKMORTWZ\\_bdfilnqtvx{}\x80\x83\x85\x88\x8a\x8d\x8f\x92\x95\x97\x9a\x9d\x9e\xa1\xa4\xa6\xa9\xac\xae\xb1\xb3\xb6\xb8\xbb\xbe\xc0\xc3\xc5\xc7\xca\xcd\xcf\xd2\xd5\xd7\xd9\xdc\xdf\xe1\xe4\xe7\xe9\xec\xee\xf0\xf3\xf6\